In [1]:
import os

# home_dir = '/content/gdrive/My Drive/AChina' 
# home_dir = '/hy-tmp'
# os.chdir(home_dir)
# pwd

!pip install tqdm 
!pip install fire

from glob import glob
from tqdm import tqdm

import numpy as np
import pandas as pd

import os
import warnings
import sys

import fire

# warnings.filterwarnings("ignore")  # avoid printing out absolute paths

# os.chdir("../../..")

!pip install scipy
!pip install torch 
!pip install pytorch_lightning==1.9.0 pytorch_forecasting

import copy
from pathlib import Path
import warnings

import time

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAPE, SMAPE, PoissonLoss, QuantileLoss
from matplotlib import pyplot as plt
from pytorch_lightning.utilities.model_summary import summarize
from pytorch_forecasting import TemporalFusionTransformer
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from matplotlib import pyplot as plt

from multiprocessing import Pool, freeze_support

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
home_dir = '/hy-tmp'        
os.chdir(home_dir)

!pwd

!ls

# from google.colab import drive
# drive.mount('/content/gdrive/')

# !ls

# home_dir = '/content/gdrive/My Drive/AChina'        
# os.chdir(home_dir)
# !pwd

/hy-tmp
ali_rice.ckpt  china  data


In [3]:
scrop = 'rice'

print( time.asctime( time.localtime(time.time()) ) )
alidata = pd.read_csv(f'data/ALIDATASET_{scrop}.csv')
print('DATASET loaded', time.asctime( time.localtime(time.time()) ) )

alidata['county'] = alidata['county'].astype(str)
alidata['year']   = alidata['year'].astype(str)

display(alidata)

Mon Feb 27 22:42:10 2023
DATASET loaded Mon Feb 27 22:43:39 2023


,Unnamed: 0,time_idx,county,year,month,MOD_month_t,b0b0,b0b1,b0b2,b0b3,...,Wind_f_tavg_254,rice_sownarea,avg_rice_sownarea,med_rice_sownarea,rice_yieldval,avg_rice_yieldval,med_rice_yieldval,rice_yield,avg_rice_yield,med_rice_yield
0,0,0,0,2003,1,0,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.9,0.45,0.22,1.8012,0.84,0.74
1,1,1,0,2003,1,1,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.9,0.45,0.22,1.8012,0.84,0.74
2,2,2,0,2003,1,2,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.9,0.45,0.22,1.8012,0.84,0.74
3,3,3,0,2003,1,3,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.9,0.45,0.22,1.8012,0.84,0.74
4,4,4,0,2003,2,0,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.9,0.45,0.22,1.8012,0.84,0.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24160,24160,40,30,2022,11,3,0.0,0.0,0.0,1.505150,...,0.0,9999.90,75.13,74.54,9999999.9,62.83,65.31,1000.0100,0.83,0.87
24161,24161,41,30,2022,12,0,0.0,0.0,0.0,0.845098,...,0.0,9999.90,75.13,74.54,9999999.9,62.83,65.31,1000.0100,0.83,0.87
24162,24162,42,30,2022,12,1,0.0,0.0,0.0,1.602060,...,0.0,9999.90,75.13,74.54,9999999.9,62.83,65.31,1000.0100,0.83,0.87
24163,24163,43,30,2022,12,2,0.0,0.0,0.0,1.301030,...,0.0,9999.90,75.13,74.54,9999999.9,62.83,65.31,1000.0100,0.83,0.87


In [5]:
aliv = alidata[ alidata['year'] == '2004']
alio = alidata[ alidata['year'] == '2003']

months = alidata['month'].unique()

for month in months:
    display(aliv[ aliv['month'] == month ])
    display(alio[ alio['month'] == month ])
               
fn

,Unnamed: 0,time_idx,county,year,month,MOD_month_t,b0b0,b0b1,b0b2,b0b3,...,Wind_f_tavg_254,rice_sownarea,avg_rice_sownarea,med_rice_sownarea,rice_yieldval,avg_rice_yieldval,med_rice_yieldval,rice_yield,avg_rice_yield,med_rice_yield
45,1215,0,0,2004,1,0,0.0,0.0,0.0,0.000000,...,0.0,0.82,0.44,0.26,1.01,0.45,0.22,1.2317,0.84,0.74
46,1216,1,0,2004,1,1,0.0,0.0,0.0,0.000000,...,0.0,0.82,0.44,0.26,1.01,0.45,0.22,1.2317,0.84,0.74
47,1217,2,0,2004,1,2,0.0,0.0,0.0,0.000000,...,0.0,0.82,0.44,0.26,1.01,0.45,0.22,1.2317,0.84,0.74
48,1218,3,0,2004,1,3,0.0,0.0,0.0,0.000000,...,0.0,0.82,0.44,0.26,1.01,0.45,0.22,1.2317,0.84,0.74
940,1259,0,1,2004,1,0,0.0,0.0,0.0,0.000000,...,0.0,13.69,19.90,18.00,11.09,15.48,12.92,0.8101,0.76,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22423,2318,3,29,2004,1,3,0.0,0.0,0.0,0.000000,...,0.0,64.38,77.00,79.27,52.46,64.09,66.46,0.8148,0.83,0.84
23315,2359,0,30,2004,1,0,0.0,0.0,0.0,1.602060,...,0.0,66.77,75.13,74.54,39.31,62.83,65.31,0.5887,0.83,0.87
23316,2360,1,30,2004,1,1,0.0,0.0,0.0,0.954243,...,0.0,66.77,75.13,74.54,39.31,62.83,65.31,0.5887,0.83,0.87
23317,2361,2,30,2004,1,2,0.0,0.0,0.0,0.301030,...,0.0,66.77,75.13,74.54,39.31,62.83,65.31,0.5887,0.83,0.87


,Unnamed: 0,time_idx,county,year,month,MOD_month_t,b0b0,b0b1,b0b2,b0b3,...,Wind_f_tavg_254,rice_sownarea,avg_rice_sownarea,med_rice_sownarea,rice_yieldval,avg_rice_yieldval,med_rice_yieldval,rice_yield,avg_rice_yield,med_rice_yield
0,0,0,0,2003,1,0,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
1,1,1,0,2003,1,1,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
2,2,2,0,2003,1,2,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
3,3,3,0,2003,1,3,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
895,45,0,1,2003,1,0,0.0,0.0,0.0,0.000000,...,0.0,7.00,19.90,18.00,5.65,15.48,12.92,0.8071,0.76,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22378,1128,3,29,2003,1,3,0.0,0.0,0.0,0.000000,...,0.0,46.71,77.00,79.27,37.04,64.09,66.46,0.7930,0.83,0.84
23270,1170,0,30,2003,1,0,0.0,0.0,0.0,1.491362,...,0.0,67.16,75.13,74.54,50.73,62.83,65.31,0.7554,0.83,0.87
23271,1171,1,30,2003,1,1,0.0,0.0,0.0,1.342423,...,0.0,67.16,75.13,74.54,50.73,62.83,65.31,0.7554,0.83,0.87
23272,1172,2,30,2003,1,2,0.0,0.0,0.0,1.361728,...,0.0,67.16,75.13,74.54,50.73,62.83,65.31,0.7554,0.83,0.87


,Unnamed: 0,time_idx,county,year,month,MOD_month_t,b0b0,b0b1,b0b2,b0b3,...,Wind_f_tavg_254,rice_sownarea,avg_rice_sownarea,med_rice_sownarea,rice_yieldval,avg_rice_yieldval,med_rice_yieldval,rice_yield,avg_rice_yield,med_rice_yield
49,1219,4,0,2004,2,0,0.0,0.0,0.0,0.000000,...,0.0,0.82,0.44,0.26,1.01,0.45,0.22,1.2317,0.84,0.74
50,1220,5,0,2004,2,1,0.0,0.0,0.0,0.000000,...,0.0,0.82,0.44,0.26,1.01,0.45,0.22,1.2317,0.84,0.74
51,1221,6,0,2004,2,2,0.0,0.0,0.0,0.000000,...,0.0,0.82,0.44,0.26,1.01,0.45,0.22,1.2317,0.84,0.74
52,1222,7,0,2004,2,3,0.0,0.0,0.0,0.000000,...,0.0,0.82,0.44,0.26,1.01,0.45,0.22,1.2317,0.84,0.74
944,1263,4,1,2004,2,0,0.0,0.0,0.0,0.000000,...,0.0,13.69,19.90,18.00,11.09,15.48,12.92,0.8101,0.76,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22427,2322,7,29,2004,2,3,0.0,0.0,0.0,0.000000,...,0.0,64.38,77.00,79.27,52.46,64.09,66.46,0.8148,0.83,0.84
23319,2363,4,30,2004,2,0,0.0,0.0,0.0,0.000000,...,0.0,66.77,75.13,74.54,39.31,62.83,65.31,0.5887,0.83,0.87
23320,2364,5,30,2004,2,1,0.0,0.0,0.0,0.477121,...,0.0,66.77,75.13,74.54,39.31,62.83,65.31,0.5887,0.83,0.87
23321,2365,6,30,2004,2,2,0.0,0.0,0.0,0.000000,...,0.0,66.77,75.13,74.54,39.31,62.83,65.31,0.5887,0.83,0.87


,Unnamed: 0,time_idx,county,year,month,MOD_month_t,b0b0,b0b1,b0b2,b0b3,...,Wind_f_tavg_254,rice_sownarea,avg_rice_sownarea,med_rice_sownarea,rice_yieldval,avg_rice_yieldval,med_rice_yieldval,rice_yield,avg_rice_yield,med_rice_yield
4,4,4,0,2003,2,0,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
5,5,5,0,2003,2,1,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
6,6,6,0,2003,2,2,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
7,7,7,0,2003,2,3,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
899,49,4,1,2003,2,0,0.0,0.0,0.0,0.000000,...,0.0,7.00,19.90,18.00,5.65,15.48,12.92,0.8071,0.76,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22382,1132,7,29,2003,2,3,0.0,0.0,0.0,0.000000,...,0.0,46.71,77.00,79.27,37.04,64.09,66.46,0.7930,0.83,0.84
23274,1174,4,30,2003,2,0,0.0,0.0,0.0,0.477121,...,0.0,67.16,75.13,74.54,50.73,62.83,65.31,0.7554,0.83,0.87
23275,1175,5,30,2003,2,1,0.0,0.0,0.0,0.301030,...,0.0,67.16,75.13,74.54,50.73,62.83,65.31,0.7554,0.83,0.87
23276,1176,6,30,2003,2,2,0.0,0.0,0.0,0.301030,...,0.0,67.16,75.13,74.54,50.73,62.83,65.31,0.7554,0.83,0.87


,Unnamed: 0,time_idx,county,year,month,MOD_month_t,b0b0,b0b1,b0b2,b0b3,...,Wind_f_tavg_254,rice_sownarea,avg_rice_sownarea,med_rice_sownarea,rice_yieldval,avg_rice_yieldval,med_rice_yieldval,rice_yield,avg_rice_yield,med_rice_yield
53,1223,8,0,2004,3,0,0.0,0.0,0.0,0.0,...,0.0,0.82,0.44,0.26,1.01,0.45,0.22,1.2317,0.84,0.74
54,1224,9,0,2004,3,1,0.0,0.0,0.0,0.0,...,0.0,0.82,0.44,0.26,1.01,0.45,0.22,1.2317,0.84,0.74
55,1225,10,0,2004,3,2,0.0,0.0,0.0,0.0,...,0.0,0.82,0.44,0.26,1.01,0.45,0.22,1.2317,0.84,0.74
56,1226,11,0,2004,3,3,0.0,0.0,0.0,0.0,...,0.0,0.82,0.44,0.26,1.01,0.45,0.22,1.2317,0.84,0.74
948,1267,8,1,2004,3,0,0.0,0.0,0.0,0.0,...,0.0,13.69,19.90,18.00,11.09,15.48,12.92,0.8101,0.76,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22431,2326,11,29,2004,3,3,0.0,0.0,0.0,0.0,...,0.0,64.38,77.00,79.27,52.46,64.09,66.46,0.8148,0.83,0.84
23323,2367,8,30,2004,3,0,0.0,0.0,0.0,0.0,...,0.0,66.77,75.13,74.54,39.31,62.83,65.31,0.5887,0.83,0.87
23324,2368,9,30,2004,3,1,0.0,0.0,0.0,0.0,...,0.0,66.77,75.13,74.54,39.31,62.83,65.31,0.5887,0.83,0.87
23325,2369,10,30,2004,3,2,0.0,0.0,0.0,0.0,...,0.0,66.77,75.13,74.54,39.31,62.83,65.31,0.5887,0.83,0.87


...

,Unnamed: 0,time_idx,county,year,month,MOD_month_t,b0b0,b0b1,b0b2,b0b3,...,Wind_f_tavg_254,rice_sownarea,avg_rice_sownarea,med_rice_sownarea,rice_yieldval,avg_rice_yieldval,med_rice_yieldval,rice_yield,avg_rice_yield,med_rice_yield
82,1252,37,0,2004,11,0,0.0,0.0,0.0,0.000000,...,0.0,0.82,0.44,0.26,1.01,0.45,0.22,1.2317,0.84,0.74
83,1253,38,0,2004,11,1,0.0,0.0,0.0,0.000000,...,0.0,0.82,0.44,0.26,1.01,0.45,0.22,1.2317,0.84,0.74
84,1254,39,0,2004,11,2,0.0,0.0,0.0,0.000000,...,0.0,0.82,0.44,0.26,1.01,0.45,0.22,1.2317,0.84,0.74
977,1296,37,1,2004,11,0,0.0,0.0,0.0,0.000000,...,0.0,13.69,19.90,18.00,11.09,15.48,12.92,0.8101,0.76,0.75
978,1297,38,1,2004,11,1,0.0,0.0,0.0,0.000000,...,0.0,13.69,19.90,18.00,11.09,15.48,12.92,0.8101,0.76,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22458,2353,38,29,2004,11,1,0.0,0.0,0.0,0.000000,...,0.0,64.38,77.00,79.27,52.46,64.09,66.46,0.8148,0.83,0.84
22459,2354,39,29,2004,11,2,0.0,0.0,0.0,0.000000,...,0.0,64.38,77.00,79.27,52.46,64.09,66.46,0.8148,0.83,0.84
23352,2396,37,30,2004,11,0,0.0,0.0,0.0,1.079181,...,0.0,66.77,75.13,74.54,39.31,62.83,65.31,0.5887,0.83,0.87
23353,2397,38,30,2004,11,1,0.0,0.0,0.0,0.954243,...,0.0,66.77,75.13,74.54,39.31,62.83,65.31,0.5887,0.83,0.87


,Unnamed: 0,time_idx,county,year,month,MOD_month_t,b0b0,b0b1,b0b2,b0b3,...,Wind_f_tavg_254,rice_sownarea,avg_rice_sownarea,med_rice_sownarea,rice_yieldval,avg_rice_yieldval,med_rice_yieldval,rice_yield,avg_rice_yield,med_rice_yield
37,37,37,0,2003,11,0,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
38,38,38,0,2003,11,1,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
39,39,39,0,2003,11,2,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
40,40,40,0,2003,11,3,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
932,82,37,1,2003,11,0,0.0,0.0,0.0,0.000000,...,0.0,7.00,19.90,18.00,5.65,15.48,12.92,0.8071,0.76,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22415,1165,40,29,2003,11,3,0.0,0.0,0.0,0.000000,...,0.0,46.71,77.00,79.27,37.04,64.09,66.46,0.7930,0.83,0.84
23307,1207,37,30,2003,11,0,0.0,0.0,0.0,0.698970,...,0.0,67.16,75.13,74.54,50.73,62.83,65.31,0.7554,0.83,0.87
23308,1208,38,30,2003,11,1,0.0,0.0,0.0,1.255273,...,0.0,67.16,75.13,74.54,50.73,62.83,65.31,0.7554,0.83,0.87
23309,1209,39,30,2003,11,2,0.0,0.0,0.0,0.778151,...,0.0,67.16,75.13,74.54,50.73,62.83,65.31,0.7554,0.83,0.87


,Unnamed: 0,time_idx,county,year,month,MOD_month_t,b0b0,b0b1,b0b2,b0b3,...,Wind_f_tavg_254,rice_sownarea,avg_rice_sownarea,med_rice_sownarea,rice_yieldval,avg_rice_yieldval,med_rice_yieldval,rice_yield,avg_rice_yield,med_rice_yield
85,1255,40,0,2004,12,0,0.0,0.0,0.0,0.000000,...,0.0,0.82,0.44,0.26,1.01,0.45,0.22,1.2317,0.84,0.74
86,1256,41,0,2004,12,1,0.0,0.0,0.0,0.000000,...,0.0,0.82,0.44,0.26,1.01,0.45,0.22,1.2317,0.84,0.74
87,1257,42,0,2004,12,2,0.0,0.0,0.0,0.000000,...,0.0,0.82,0.44,0.26,1.01,0.45,0.22,1.2317,0.84,0.74
88,1258,43,0,2004,12,3,0.0,0.0,0.0,0.000000,...,0.0,0.82,0.44,0.26,1.01,0.45,0.22,1.2317,0.84,0.74
980,1299,40,1,2004,12,0,0.0,0.0,0.0,0.000000,...,0.0,13.69,19.90,18.00,11.09,15.48,12.92,0.8101,0.76,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22463,2358,43,29,2004,12,3,0.0,0.0,0.0,0.000000,...,0.0,64.38,77.00,79.27,52.46,64.09,66.46,0.8148,0.83,0.84
23355,2399,40,30,2004,12,0,0.0,0.0,0.0,1.361728,...,0.0,66.77,75.13,74.54,39.31,62.83,65.31,0.5887,0.83,0.87
23356,2400,41,30,2004,12,1,0.0,0.0,0.0,1.568202,...,0.0,66.77,75.13,74.54,39.31,62.83,65.31,0.5887,0.83,0.87
23357,2401,42,30,2004,12,2,0.0,0.0,0.0,1.000000,...,0.0,66.77,75.13,74.54,39.31,62.83,65.31,0.5887,0.83,0.87


,Unnamed: 0,time_idx,county,year,month,MOD_month_t,b0b0,b0b1,b0b2,b0b3,...,Wind_f_tavg_254,rice_sownarea,avg_rice_sownarea,med_rice_sownarea,rice_yieldval,avg_rice_yieldval,med_rice_yieldval,rice_yield,avg_rice_yield,med_rice_yield
41,41,41,0,2003,12,0,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
42,42,42,0,2003,12,1,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
43,43,43,0,2003,12,2,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
44,44,44,0,2003,12,3,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
936,86,41,1,2003,12,0,0.0,0.0,0.0,0.000000,...,0.0,7.00,19.90,18.00,5.65,15.48,12.92,0.8071,0.76,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22419,1169,44,29,2003,12,3,0.0,0.0,0.0,0.000000,...,0.0,46.71,77.00,79.27,37.04,64.09,66.46,0.7930,0.83,0.84
23311,1211,41,30,2003,12,0,0.0,0.0,0.0,1.361728,...,0.0,67.16,75.13,74.54,50.73,62.83,65.31,0.7554,0.83,0.87
23312,1212,42,30,2003,12,1,0.0,0.0,0.0,0.477121,...,0.0,67.16,75.13,74.54,50.73,62.83,65.31,0.7554,0.83,0.87
23313,1213,43,30,2003,12,2,0.0,0.0,0.0,1.491362,...,0.0,67.16,75.13,74.54,50.73,62.83,65.31,0.7554,0.83,0.87


NameError: name 'fn' is not defined

In [ ]:
infer_mask = alidata['year'].isin(['2019', '2020', '2021', '2022'])

data_infer = alidata[infer_mask]

data_infer['rice_sownarea'] = np.nan
data_infer['rice_yieldval'] = np.nan
data_infer['rice_yield']    = np.nan

years = [str(x) for x in range(2003, 2005)]

val_year = str(2004)

years.remove(val_year)

train_mask = alidata['year'].isin(years)
data = alidata[train_mask]

val_mask = alidata['year'].isin([val_year])
data_val = alidata[val_mask]

display(data_infer)

display(data)

display(data_val)


In [ ]:
max_prediction_length = 16
max_encoder_length = 27
training_cutoff = data["time_idx"].max() - max_prediction_length

avg_med = ["avg_rice_yield", "med_rice_yield", "avg_rice_sownarea", "med_rice_sownarea",\
                 "avg_rice_yieldval", "med_rice_yieldval"]

_static_reals = avg_med

_time_varying_known_reals = ["time_idx"]
_time_varying_known_reals.extend(avg_med)

# display( data[ [ col for col in _time_varying_known_reals ] ] )

# fn

mod_names = [f'b{iband}b{bins}' for iband in range(9) for bins in range(512)]

famine_list = ['Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'Psurf_f_tavg', 'Qair_f_tavg', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg',\
            'Qs_tavg', 'Qsb_tavg', 'RadT_tavg', 'Rainf_f_tavg', 'SnowCover_inst', 'SnowDepth_inst', 'Snowf_tavg', \
            'SoilMoi00_10cm_tavg', 'SoilMoi10_40cm_tavg', 'SoilMoi40_100cm_tavg', \
            'SoilTemp00_10cm_tavg', 'SoilTemp10_40cm_tavg', 'SoilTemp40_100cm_tavg', \
            'SWdown_f_tavg', 'SWE_inst', 'Swnet_tavg', 'Tair_f_tavg', 'Wind_f_tavg']

nbins = ['_' + str(x) for x in range(0, 255)]

famine_names = [famine + bb for famine in famine_list for bb in nbins]

_time_varying_unknown_reals = []
_time_varying_unknown_reals.extend(avg_med)
_time_varying_unknown_reals.extend(mod_names)
_time_varying_unknown_reals.extend(famine_names)

# display( data[ [ col for col in _time_varying_unknown_reals ] ] )

# fn

# display( data[ [ col for col in mod_names ] ] )

# display( data[ [ col for col in famine_names ] ] )

# fn

print( time.asctime( time.localtime(time.time()) ) )

training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="rice_yield",
    group_ids=["county", "year"],
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length = 1,
    max_prediction_length = max_prediction_length,
    # static_categoricals = ["county", "years"],
    # static_reals = _static_reals,
    # time_varying_known_categoricals=["special_days", "month"],
    # variable_groups={"special_days": special_days},  # group of categorical variables can be treated as one variable
    time_varying_known_reals = _time_varying_known_reals,
    # time_varying_unknown_categoricals=[],
    time_varying_unknown_reals = _time_varying_unknown_reals,
    # target_normalizer=GroupNormalizer(
    #     groups=["agency", "sku"], transformation="softplus"
    # ),  # use softplus and normalize by group
    # add_relative_time_idx=True,
    # add_target_scales=True,
    # add_encoder_length=True,
)

print( time.asctime( time.localtime(time.time()) ) )

validation = TimeSeriesDataSet(
    data_val[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="rice_yield",
    group_ids=["county", "year"],
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length = 1,
    max_prediction_length = max_prediction_length,
    # static_categoricals = ["county", "years"],
    # static_reals = _static_reals,
    # time_varying_known_categoricals=["special_days", "month"],
    # variable_groups={"special_days": special_days},  # group of categorical variables can be treated as one variable
    time_varying_known_reals = _time_varying_known_reals,
    # time_varying_unknown_categoricals=[],
    time_varying_unknown_reals = _time_varying_unknown_reals,
    # target_normalizer=GroupNormalizer(
    #     groups=["agency", "sku"], transformation="softplus"
    # ),  # use softplus and normalize by group
    # add_relative_time_idx=True,
    # add_target_scales=True,
    # add_encoder_length=True,
)

# create validation set (predict=True) which means to predict the last max_prediction_length points in time
# for each series
# validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)

print( time.asctime( time.localtime(time.time()) ) )


In [ ]:
print(data['b0b10'].dtypes)
print(data['Evap_tavg_10'].dtypes)

In [ ]:
# create dataloaders for model
batch_size = 64  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=8)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=8)

In [ ]:
print( time.asctime( time.localtime(time.time()) ) )
# calculate baseline mean absolute error, i.e. predict next value as the last available value from the history
actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
(actuals - baseline_predictions).abs().mean().item()
print('Baseline:', time.asctime( time.localtime(time.time()) ) )

In [ ]:
dir = '/hy-tmp/chck/ali'
dir = os.path.join(home_dir, 'data')
checkpoint_callback = ModelCheckpoint(dirpath=dir + 'ali', every_n_epochs=1)

dir = '/tf_logs'
dir = os.path.join(home_dir, 'data')
logger = TensorBoardLogger(dir, name='ali', comment='ali')

lr_monitor = LearningRateMonitor(logging_interval='epoch')

trainer = Trainer(accelerator='gpu', logger=logger, log_every_n_steps=1, fast_dev_run=True, max_epochs=30, \
                  callbacks=[checkpoint_callback, lr_monitor])

learning_rate = 0.001

model = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=learning_rate,
    # # lstm_layers=2,
    hidden_size=16,
    # attention_head_size=4,
    # dropout=0.1,
    # hidden_continuous_size=8,
    # output_size=7,  # 7 quantiles by default
    # loss=RMSE(),
    loss=QuantileLoss(),
    # log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    # reduce_on_plateau_patience=4,
    )

In [ ]:
print( time.asctime( time.localtime(time.time()) ) )
trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)
print('fit:', time.asctime( time.localtime(time.time()) ) )

In [ ]:
# load the best model according to the validation loss
# (given that we use early stopping, this is not necessarily the last epoch)
# os.chdir(os.path.join(home_dir, 'data'))
#   best_model_path = trainer.checkpoint_callback.best_model_path
# print(type(best_model_path))
# best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

print('weights saving', time.asctime( time.localtime(time.time()) ) )
checkpoint = f"ali_{scrop}.ckpt"
trainer.save_checkpoint(checkpoint)
print('weights loading', time.asctime( time.localtime(time.time()) ) )
best_tft = model  # TemporalFusionTransformer.load_from_checkpoint(checkpoint)
print('weights loaded', time.asctime( time.localtime(time.time()) ) )

In [ ]:
# calcualte mean absolute error on validation set
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions = best_tft.predict(val_dataloader)
(actuals - predictions).abs().mean()

In [ ]:
print(type(actuals), actuals.device, actuals.size(), actuals.size()[0])
print(type(predictions), predictions.device, predictions.size())

print(actuals[0,0], predictions[0,0])

for county in range(actuals.size()[0]):
    print(actuals[county,:], predictions[county,:])
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20,5))
    ax.plot(actuals[county, :], color='r')
    ax.plot(predictions[county, :], color='b')

plt.show()

In [ ]:
# raw predictions are a dictionary from which all kind of information including quantiles can be extracted
raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)

In [ ]:
print(type(raw_predictions))

In [ ]:
for idx in range(10):  # plot 10 examples
    best_tft.plot_prediction(x, raw_predictions, idx=idx, add_loss_to_title=True);